In [ ]:
# for logistic regression
class Logistic_Regression():
    def __init__(self, x_train, y_train, x_test, y_test, classes, learning_rate):
        self.x_train, self.y_train, self.x_test, self.y_test = x_train, y_train, x_test, y_test
        self.classes = classes
        self.eta = learning_rate
        
        # initial weight to be 0 everywhere
        #self.W = np.random.uniform(size=(classes, x_train.shape[1]))
        self.W = np.zeros((self.classes, x_train.shape[1]))
        
        # learning history
        self.train_loss, self.train_precision = [], []
        self.test_loss, self.test_precision = [], []
    
    # calculate the prediction
    def Prediction(self):
        self.x_train_pred = np.dot(self.x_train, self.W.T)
        self.x_test_pred = np.dot(self.x_test, self.W.T)
    
    # use softmax to normalize the probability
    def Softmax(self):
        # training set
        exp_z = np.exp(self.x_train_pred)
        exp_sum = np.sum(exp_z, axis=1).reshape(exp_z.shape[0], 1)
        self.train_pred = exp_z / exp_sum
        
        # test set
        exp_z = np.exp(self.x_test_pred)
        exp_sum = np.sum(exp_z, axis=1).reshape(exp_z.shape[0], 1)
        self.test_pred = exp_z / exp_sum
    
    # esitmation prediction perfomance
    def Cross_entropy(self):
        # training set
        n = self.y_train.shape[0]
        error = 0
        q = np.log2(self.train_pred)
        for i in range(n):
            for j in range(self.classes):
                error -= self.y_train[i][j]*q[i][j]
        self.train_loss.append(error / n)
        
        # test set
        n = self.y_test.shape[0]
        error = 0
        q = np.log2(self.test_pred)
        for i in range(n):
            for j in range(self.classes):
                error -= self.y_test[i][j]*q[i][j]
        self.test_loss.append(error / n)
    
    # calculate precision
    def Precision(self):
        # training set
        one_hot_pred = np.argmax(self.train_pred, axis=1)
        one_hot_y = np.argmax(self.y_train, axis=1)
        truth = one_hot_pred.shape[0]
        count = truth
        for i in range(truth):
            if one_hot_pred[i] != one_hot_y[i]:
                count -= 1
        self.train_precision.append(count / truth)
        
        # test set
        one_hot_pred = np.argmax(self.test_pred, axis=1)
        one_hot_y = np.argmax(self.y_test, axis=1)
        truth = one_hot_pred.shape[0]
        count = truth
        for i in range(truth):
            if one_hot_pred[i] != one_hot_y[i]:
                count -= 1
        self.test_precision.append(count / truth)
    
    # back-propagation of (softmax + cross-entropy)
    def Derivative_cross_entropy(self, y_pred, y):
        self.derivative = y_pred - y
    
    # batch gradient descent
    def Batch_GD(self):
        self.Derivative_cross_entropy(self.train_pred, self.y_train)
        gradient = np.dot(self.derivative.T, self.x_train)
        self.W -= gradient * self.eta
    
    # stochastic gradient descent, with 32 iteration for each epoch
    def SGD(self):
        for i in range(32):
            id = np.random.randint(self.y_train.shape[0], size=1)
            self.Derivative_cross_entropy(self.train_pred[id], self.y_train[id])
            gradient = np.dot(self.derivative.T, self.x_train[id])
            self.W -= gradient * self.eta
    
    # minibatch gradient descent, with user-defined minibatch
    def Minibatch_SGD(self):
        id = np.random.choice(self.y_train.shape[0], self.minibatch, replace=False)

        sgd_x = np.zeros((self.minibatch, self.x_train.shape[1]))
        sgd_y = np.zeros((self.minibatch, self.classes))
        sgd_y_pred = np.zeros((self.minibatch, self.classes))
        for i in range(self.minibatch):
            sgd_x[i, :] = self.x_train[id[i], :]
            sgd_y[i, :] = self.y_train[id[i], :]
            sgd_y_pred[i, :] = self.train_pred[id[i], :]
        
        self.Derivative_cross_entropy(sgd_y_pred, sgd_y)
        gradient = np.dot(self.derivative.T, sgd_x)
        self.W -= gradient * self.eta
    
    # plot learning curve
    def Plot(self):
        x_axis = list(range(self.epoch))
        # plot loss curve
        plt.subplot(1, 2, 1)
        plt.plot(x_axis, self.train_loss, color='blue', label='training loss')
        plt.plot(x_axis, self.test_loss, color='orange', label='testing loss')
        plt.legend()
        
        # plot precision curve
        plt.subplot(1, 2, 2)
        percetage_train = [round(i, 2)*100 for i in self.train_precision]
        percetage_test = [round(i, 2)*100 for i in self.test_precision]
        plt.plot(x_axis, percetage_train, color='blue', label='training accuracy')
        plt.plot(x_axis, percetage_test, color='orange', label='testing accuracy')
        plt.legend()

        plt.show()
    
    def Display(self):
        print('for training data, the final classification accuracy = '+str(round(self.train_precision[self.epoch-1], 5))+
              ', and loss = '+str(round(self.train_loss[self.epoch-1], 5)))
        print('for testing  data, the final classification accuracy = '+str(round(self.test_precision[self.epoch-1], 5))+
              ', and loss = '+str(round(self.test_loss[self.epoch-1], 5)))
    
    # process logistic regression
    def Training(self, epoch, optimizer, minibatch=32):
        self.epoch = epoch
        self.optimizer = optimizer
        self.minibatch = minibatch
        
        for i in range(epoch):
            # model prediction
            self.Prediction()
            
            # normalize probability
            self.Softmax()
            
            # estimation prediction performance
            self.Cross_entropy()
            
            # calculate precision
            self.Precision()

            # back-propagation with different method
            if self.optimizer == 'Batch GD':
                self.Batch_GD()
            elif self.optimizer == 'SGD':
                self.SGD()
            elif self.optimizer == 'Minibatch SGD':
                self.Minibatch_SGD()
        
        # plot learning curve with loss and accuracy
        print('For ' + self.optimizer + ' :')
        self.Plot()
        
        # display final information
        self.Display()